In [1]:
# Source: https://github.com/carlosjsaez/MultiClassBERT/blob/main/BERT_Multi_Class_for_Scoring_Classification.ipynb

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_pickle("./data/final_cleaned_submissions.pkl")
data = data[["text", "sentiment"]]#[:5000]

In [3]:
def concatenate_words_in_sentence(text):
    new_text = []
    s = ""
    for sentence in text:
        for word in sentence:
            s = s + word + " "
        new_text.append(s)
        s = ""
    return new_text

In [4]:
data["new_text"] = concatenate_words_in_sentence(data["text"].tolist())

In [5]:
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification

In [6]:
possible_labels = data.sentiment.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'bullish': 0, 'neutral': 1, 'bearish': 2}

In [7]:
data['label'] = data.sentiment.replace(label_dict)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_val_i, y_train, y_val_i = train_test_split(data.index.values, 
                                                  data.sentiment.values, 
                                                  test_size=0.4,
                                                  stratify=data.sentiment.values)

X_val, X_test, y_val, y_test = train_test_split(X_val_i, 
                                                  y_val_i, 
                                                  test_size=0.5,
                                                  stratify=y_val_i)

data['data_type'] = ['not_set']*data.shape[0]

data.loc[X_train, 'data_type'] = 'train'
data.loc[X_val, 'data_type'] = 'val'
data.loc[X_test, 'data_type'] = 'test'

data.groupby(['sentiment', 'label', 'data_type']).count()

text  new_text
sentiment label data_type                 
bearish   2     test        4818      4818
                train      14453     14453
                val         4817      4817
bullish   0     test       24223     24223
                train      72667     72667
                val        24223     24223
neutral   1     test        6868      6868
                train      20606     20606
                val         6869      6869

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

max_length = 64

encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type=='train'].new_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=max_length, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    data[data.data_type=='val'].new_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=max_length, 
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    data[data.data_type=='test'].new_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=max_length, 
    return_tensors='pt'
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\DEV\Master Thesis\App\.thesis\lib\site-packages\transformers\tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [10]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data[data.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data[data.data_type=='val'].label.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(data[data.data_type=='test'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [11]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 2

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [13]:
from transformers import AdamW, Adafactor, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5,
                  eps=1e-8)

# optimizer = Adafactor(model.parameters(),
#                  lr = 1e-6,
#                  relative_step = False)

epochs = 3

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [14]:
from sklearn.metrics import f1_score, accuracy_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return accuracy_score(labels_flat, preds_flat)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')


In [15]:
import random
import numpy as np

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    
    return loss_val_avg, predictions, true_vals

In [16]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [17]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [18]:
acc_list = []
val_acc_list = []
loss_list = []
val_loss_list = []

for epoch in tqdm(range(1, epochs+1)):

    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()


        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


    torch.save(model.state_dict(), f'data/finetuned_BERT_epoch_{epoch}.model')

    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    # Train Accuracy
    t_loss, t_predictions, t_vals = evaluate(dataloader_train) #DEV
    t_acc = accuracy_score_func(t_predictions, t_vals) #DEV
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    val_acc = accuracy_score_func(predictions, true_vals)
    
    tqdm.write(f'Dev Train Loss: {t_loss}') #DEV
    tqdm.write(f'Dev Train Acc: {t_acc}') #DEV


    tqdm.write(f'Validation loss: {val_loss}')
    #tqdm.write(f'F1 Score (Weighted): {val_f1}')
    tqdm.write(f'Val Accuracy: {val_acc}')

    acc_list.append(t_acc) #DEV
    loss_list.append(loss_train_avg)

    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/53863 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.694658406827971
Dev Train Loss: 0.4446741451594571
Dev Train Acc: 0.8970443532666209
Validation loss: 0.5885427629519633
Val Accuracy: 0.8682781475396141


Epoch 2:   0%|          | 0/53863 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.4859022303576887
Dev Train Loss: 0.30518312880205045
Dev Train Acc: 0.9413326402168465
Validation loss: 0.5944069210711321
Val Accuracy: 0.8897490879723746


Epoch 3:   0%|          | 0/53863 [00:00<?, ?it/s]


Epoch 3
Training loss: 0.34117712252717414
Dev Train Loss: 0.22467992277243992
Dev Train Acc: 0.9623303566455638
Validation loss: 0.6442386929535986
Val Accuracy: 0.8971845498343034


In [19]:
print("Train Loss:", loss_list)
print("Val Loss:", val_loss_list)
print()
print("Train Acc:", acc_list)
print("Val Acc:", val_acc_list)

Train Loss: [0.694658406827971, 0.4859022303576887, 0.34117712252717414]
Val Loss: [0.5885427629519633, 0.5944069210711321, 0.6442386929535986]

Train Acc: [0.8970443532666209, 0.9413326402168465, 0.9623303566455638]
Val Acc: [0.8682781475396141, 0.8897490879723746, 0.8971845498343034]


In [20]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('data/finetuned_BERT_epoch_3.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_test)
accuracy_per_class(predictions, true_vals)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Class: bullish
Accuracy: 22876/24223

Class: neutral
Accuracy: 5288/6868

Class: bearish
Accuracy: 4023/4818



In [21]:
converted_test_pred = []
for i in range(len(predictions)):
    maxi = np.argmax(predictions[i])
    arr = np.zeros((3))
    arr[maxi] = 1
    converted_test_pred.append(arr)

In [22]:
new_converted_test_pred = []
for item in converted_test_pred:
    if item[0] == 1:
        new_converted_test_pred.append(0)
    elif item[1] == 1:
        new_converted_test_pred.append(1)
    elif item[2] == 1:
        new_converted_test_pred.append(2)

In [23]:
from sklearn.metrics import classification_report
print(classification_report(true_vals, new_converted_test_pred))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93     24223
           1       0.84      0.77      0.80      6868
           2       0.83      0.83      0.83      4818

    accuracy                           0.90     35909
   macro avg       0.87      0.85      0.86     35909
weighted avg       0.90      0.90      0.90     35909



In [24]:
from sklearn.metrics import accuracy_score
print(accuracy_score(true_vals, new_converted_test_pred))

0.8963491046812777


In [42]:
# Eval:
drei = torch.load('data/finetuned_BERT_epoch_3.model')